# 🐌 Prepare imports

## 🦛 Install need packages

In [1]:
! pip install gdown

In [2]:
import gdown

url = "https://drive.google.com/uc?id=15uPcQurzdz2aO2Bgf3WLEsQ3nAaMaM8T"
output = 'requirements.txt'
gdown.download(url, output, quiet=False);

Downloading...
From: https://drive.google.com/uc?id=15uPcQurzdz2aO2Bgf3WLEsQ3nAaMaM8T
To: /kaggle/working/requirements.txt
100%|██████████| 2.36k/2.36k [00:00<00:00, 6.73MB/s]


In [3]:
! pip install -r requirements.txt
! pip install tianshou coloredlogs
! pip install --upgrade ipython

  Cloning https://github.com/thu-ml/tianshou.git (to revision ade85ab32baab721605508dfd9d460015a1832e5) to /tmp/pip-install-khjs8b2z/tianshou_db26d01f0cf14a5c90423b974baf9d3e
  Running command git clone --filter=blob:none --quiet https://github.com/thu-ml/tianshou.git /tmp/pip-install-khjs8b2z/tianshou_db26d01f0cf14a5c90423b974baf9d3e
  Running command git rev-parse -q --verify 'sha^ade85ab32baab721605508dfd9d460015a1832e5'
  Running command git fetch -q https://github.com/thu-ml/tianshou.git ade85ab32baab721605508dfd9d460015a1832e5
  Running command git checkout -q ade85ab32baab721605508dfd9d460015a1832e5
  Resolved https://github.com/thu-ml/tianshou.git to commit ade85ab32baab721605508dfd9d460015a1832e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 

In [4]:
# ! pip install -U ipywidgets

## 🐦 Imports

In [5]:
import gymnasium
import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete
from gymnasium import spaces
import random

from IPython.display import clear_output
import time
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers
import matplotlib.pyplot as plt
from tianshou.env.pettingzoo_env import PettingZooEnv

from typing import Optional, Tuple
from tianshou.policy import BasePolicy, DQNPolicy, RainbowPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import OffpolicyTrainer
from tianshou.utils.net.common import Net
from tianshou.env import DummyVectorEnv
from copy import deepcopy
from tianshou.data import Collector, VectorReplayBuffer

from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger

import torch

import random
import os

NUM_ITERS = 400
PLAYS = {"bastaushy": 0, "qostaushy": 0}

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


# 🐘 AEC environment https://pettingzoo.farama.org/api/aec/#about-aec

## 🦉 Create environment

In [6]:
class TogyzQumalaqEnv(AECEnv):
    """
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {
        "render_modes": ["ansi", "human"],
        "name": "togyzqumalaq_v0"
        }

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - otaular
        - tuzdyq
        - qazandar
        - possible_agents
        - render_mode
        """
        self.otaular = []
        self.tuzdyq = []
        self.qazandar = []
        self.direction = []
        self.agents = ["bastaushy", "qostaushy"]
        self.possible_agents = self.agents[:]
        self.action_spaces = {i: spaces.Discrete(9) for i in self.agents}
        self.observation_spaces = {
            i: spaces.Dict(
                {
                    "observation":
                        MultiDiscrete([100] * 18 + [10] * 2 + [162] * 2 + [2]),
                    "action_mask":
                        Discrete(9),
                }
            )
            for i in self.agents
        }
        self.render_mode = render_mode

    # Action space should be defined here.
    def action_space(self, agent):
        return self.action_spaces[agent]

    # Observation space should be defined here.
    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def render(self):
        """
        Renders the environment. In human mode,
        it can print to terminal, open
        up a graphical window, or open up some
        other display that
        a human can see and understand.
        """
        """Renders the environment."""
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without "
                "specifying any render mode."
            )
            return

        if len(self.agents) == 2:
            points_bastaushy_x = np.array([i * 2 for i in range(10)])
            points_bastaushy_y = np.array([i % 5 for i in range(50)])

            qazandar = self.qazandar
            otaular = self.otaular
            tuzdyq = self.tuzdyq
            x = np.arange(-3, 225, 1)
            y = -1

            text_kwargs = dict(ha='center', va='center', fontsize=12)
            plt.figure(figsize=(15, 4))

            for i in range(9):
                # qostaushy's part
                plt.scatter(np.repeat(
                    points_bastaushy_x + 25 * i, 5)[:otaular[17 - i]],
                            points_bastaushy_y[:otaular[17 - i]], marker='o')
                # vertical lines
                plt.plot(np.repeat(25 * i - 2, len(x)),
                         np.arange(-7, 5, 12 / len(x)))
                # bastaushy's part
                plt.scatter(np.repeat(points_bastaushy_x + 25 * i, 5)[:otaular[i]],
                                points_bastaushy_y[:otaular[i]] - 6, marker='o')
            
            # horizontal line
            x_lims = np.arange(-3, 245, 1)
            plt.plot(x_lims, np.repeat(y, len(x_lims)))
            # last vertical line
            plt.plot(np.repeat(25 * 9 - 2, 13),
                     np.arange(-7, 6, 1))
        
            for i in range(9):
                # bastaushy's qumalaqtar
                plt.text(25 * i + 10, -7,
                         f'{i} ({otaular[i]})', **text_kwargs)
                # qostaushy's qumalaqtar
                plt.text(25 * i + 10, 5,
                         f'{17 - i} ({otaular[17 - i]})', **text_kwargs)
            # bastaushy qazan's qumalaqtar
            plt.text(235, -4,
                     f'qazan: {qazandar[0]}', **text_kwargs)
            # qostaushy qazan's qumalaqtar
            plt.text(235, 2,
                     f'qazan: {qazandar[1]}', **text_kwargs)
            # bastaushy tuzdyq's qumalaqtar
            plt.text(235, -6,
                     f'tuzdyq: {tuzdyq[0]}', **text_kwargs)
            # qostaushy tuzdyq's qumalaqtar
            plt.text(235, 0,
                     f'tuzdyq: {tuzdyq[1]}', **text_kwargs)
            plt.xticks([])
            plt.yticks([])
            plt.show()
        else:
            if self.render_mode == "human":
                print("Game over")
        time.sleep(1)
        #clear_output()

    def _legal_moves(self, agent):
        cur_player = self.possible_agents.index(agent)
        opp_player = (cur_player + 1) % 2
        return [item for item in range(9) if self.tuzdyq[opp_player] != item  + cur_player * 9 and self.otaular[item + cur_player * 9] > 0]

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        # observation of one agent is the previous state of the other
        legal_moves = self._legal_moves(agent) if agent == self.agent_selection else []
        action_mask = np.zeros(9, "int8")
        
        for i in legal_moves:
            action_mask[i] = 1
        observation = tuple(
            self.otaular + self.tuzdyq + self.qazandar + [self.possible_agents.index(self.agent_selection)]
        )
        return {"observation": observation, "action_mask": action_mask}

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.otaular = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
        self.direction = [list(range(18)), [9, 10, 11, 12, 13, 14, 15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8]]
        self.tuzdyq = [-1, -1]
        self.qazandar = [0, 0]
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.num_moves = 0
        observation = tuple(
            self.otaular + self.tuzdyq + self.qazandar + [0]
        )
        self.observations = {agent: observation for agent in self.agents}
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self._was_dead_step(action)
            return
        self.rewards = {agent: 0 for agent in self.agents}
        cur_player = self.possible_agents.index(self.agent_selection)
        opp_player = (cur_player + 1) % 2
        self.num_moves += 1
        if self.render_mode == "human":
            print(f'MOVE #{self.num_moves}')
        # The truncations dictionary must be updated for all players.
        self.truncations = {
            agent: self.num_moves >= NUM_ITERS for agent in self.agents
        }
        # distribute qumalaqs
        if cur_player == 1:
            action += 9
        if self.render_mode == "human":
            print(f'{self.agent_selection} made action {action}')
        num_qumalaq = self.otaular[action]
        idx_action = self.direction[cur_player].index(action)
        if self.otaular[action] == 1:
            self.otaular[self.direction[cur_player][idx_action + 1]] += 1
            self.otaular[action] -= 1
        else:
            i = 1
            coef = 1
            if self.otaular[action] / 18 > 1:
                coef = int(self.otaular[action] / 18) + 1
            while self.otaular[action] > coef:
                self.otaular[self.direction[cur_player][(idx_action + i) % 18]] += 1
                self.otaular[action] -= 1
                i += 1
        # check tuzdyq & add rewards to qazandar
        reward = 0
        if self.tuzdyq[cur_player] < 0 and self.check_tuzdyq(self.agent_selection, action, num_qumalaq):
            reward += 3
            if self.render_mode == "human":
                print(f'{self.agent_selection} won tuzdyq {reward}')
                
            #******* awarding a rewards for receiving tuzdyq **********
            # self.rewards[self.agent_selection] += 50
            # self.rewards[self.possible_agents[opp_player]] -= 50
        else:

            if num_qumalaq > 1:
                last_otau = self.direction[cur_player][(idx_action + num_qumalaq - 1) % 18]
            else:
                last_otau = self.direction[cur_player][(idx_action + num_qumalaq) % 18]

            if (last_otau in range(opp_player * 9, (opp_player + 1) * 9) and
                    self.otaular[last_otau] % 2 == 0):
                reward += self.otaular[last_otau]
                if self.render_mode == "human":
                    print(f'{self.agent_selection} won {reward}')
                self.otaular[last_otau] = 0
            if (self.tuzdyq[cur_player] >= 0 and
                    self.otaular[self.tuzdyq[cur_player]] > 0):
                reward += self.otaular[self.tuzdyq[cur_player]]
                if self.render_mode == "human":
                    print(f'{self.agent_selection} won tuzdyq {self.otaular[self.tuzdyq[cur_player]]}')
                self.otaular[self.tuzdyq[cur_player]] = 0
        if self.render_mode == "human":
            print(f'{self.agent_selection} won total {reward}')
        self.qazandar[cur_player] += reward

        #******* awarding a rewards from otaular **********
        self.rewards[self.agent_selection] += reward
        self.rewards[self.possible_agents[opp_player]] -= reward
        
        # check if there is a winner
        winner = self.check_for_winner()
        if winner:
            self.terminations = {i: True for i in self.agents}
            if self.render_mode == "human":
                print(f'{self.agent_selection} won the game!!!')
                
            #******* awarding a reward for winning a game **********
            self.rewards[self.agent_selection] += self.qazandar[opp_player]
            self.rewards[self.possible_agents[opp_player]] -= self.qazandar[opp_player]
            for i in range(9):
                self.rewards[self.agent_selection] += self.otaular[i + 9 * opp_player]
                self.rewards[self.possible_agents[opp_player]] -= self.otaular[i + 9 * opp_player]
            
        # selects the next agent.
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        total_rewards = sum(self.rewards.values())
        assert total_rewards == 0, f"Error: Total reward is not zero: {total_rewards}"
        total_qumalaqs = 0
        for i in self.otaular:
            total_qumalaqs += i
        for i in self.qazandar:
            total_qumalaqs += i
        assert total_qumalaqs == 162, f"Error: Total qumalaqs is not equal to 162: {total_qumalaqs}"
        if self.render_mode == "human":
            self.render()

    def check_tuzdyq(self, agent, action, num_qumalaq):
        cur_player = self.possible_agents.index(agent)
        opp_player = (cur_player + 1) % 2
        idx = self.direction[cur_player].index(action)

        if num_qumalaq > 1:
            last_otau = self.direction[cur_player][(idx + num_qumalaq - 1) % 18]
        else:
            last_otau = self.direction[cur_player][(idx + num_qumalaq) % 18]

        if (last_otau in range(opp_player * 9, (opp_player + 1) * 9) and
                self.otaular[last_otau] == 3 and last_otau != 17 - cur_player * 9 and
                abs(last_otau - self.tuzdyq[opp_player]) != 9):
            self.tuzdyq[cur_player] = last_otau
            self.otaular[last_otau] = 0
            if self.render_mode == "human":
                print(f'{agent} got tuzdyq {last_otau}!')
            return True

        return False

    def check_atsyrau(self, agent):
        cur_player = self.possible_agents.index(agent)
        opp_player = (cur_player + 1) % 2

        for idx, i in enumerate(
                self.otaular[cur_player * 9: (cur_player + 1) * 9]):
            if i > 0 and idx + cur_player * 9 != self.tuzdyq[opp_player]:
                return False
        if self.render_mode == "human":
            print(f'{agent} reached atsyrau')
        return True

    def check_for_winner(self):
        cur_player = self.possible_agents.index(self.agent_selection)
        opp_player = (cur_player + 1) % 2
        if self.qazandar[cur_player] > 81:
            PLAYS[self.agent_selection] += 1
            return True
        if (self.check_atsyrau(self.possible_agents[opp_player])
                and self.qazandar[opp_player] <= 81):
            PLAYS[self.agent_selection] += 1
            return True
        return False


def _get_env(render_mode=None):
    """This function is needed to provide callables for DummyVectorEnv."""
    def env(render_mode=None):
        """
        The env function often wraps the environment in wrappers by default.
        You can find full documentation for these methods
        elsewhere in the developer documentation.
        """
        internal_render_mode = render_mode \
            if render_mode != "ansi" else "human"
        env = TogyzQumalaqEnv(render_mode=internal_render_mode)
        # This wrapper is only for environments
        # which print results to the terminal
        if render_mode == "ansi":
            env = wrappers.CaptureStdoutWrapper(env)
        # this wrapper helps error handling for discrete action spaces
        env = wrappers.AssertOutOfBoundsWrapper(env)
        # Provides a wide vareity of helpful user errors
        # Strongly recommended
        env = wrappers.OrderEnforcingWrapper(env)
        return env
    return PettingZooEnv(env(render_mode=render_mode))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 🐦 Prepare data

## 🐆 Create folders

### 🦂 Log folders

In [7]:
!mkdir logs
LOGS_PATH = os.path.join('/', 'kaggle', 'working', 'logs')

### 🦆 Model folders

In [8]:
!mkdir models
MODELS_PATH = os.path.join('/', 'kaggle', 'working', 'models')

## 🐠 Training parameters

In [9]:
NET_ARCHS = [[64,128,128,64], [1024, 2048, 2048, 1024], [2048, 4096, 4096, 2048], [2048, 4096, 8192, 4096, 2048]]

MAX_EPOCHS = 4000

STEP_PER_EPOCH = 10000

STEP_PER_COLLECT = 1000

EPISODE_PER_TEST = 200

UPDATE_PER_STEP = 1 / STEP_PER_COLLECT

BATCH_SIZE = 1024

LR = 1e-04

DF = 0.99

TRAIN_EPS = 0.15

TEST_EPS = 0.05

TRAIN_NUM = 100

TEST_NUM = 10

agent_idx2name = {0: 'bastaushy', 1: 'qostaushy'}

SEED = 888

# 🐼 DQN agent to play vs a random policy agent https://pettingzoo.farama.org/tutorials/tianshou/intermediate/

## 🐜 Set net architectures

In [10]:
def arch2str(net_arch):
    out = str(net_arch[0])
    for i in net_arch[1:]:
        out += 'x' + str(i)
    return out

def str2arch(inp):
    lst = inp.split('x')
    return [int(i) for i in lst]

## 🐫 Prepare main functions¶

In [11]:
def _get_agents_dqn(
    agent_train: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
    agent_type = None,
    net_arch = [64, 128, 128, 64]
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gymnasium.spaces.Dict)
        else env.observation_space
    )
    if agent_train is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=net_arch,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )

        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=LR)
        agent_train = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=DF,
            estimation_step=3,
            target_update_freq=320,
            action_space=env.action_space,
            observation_space=observation_space
        ).to("cuda" if torch.cuda.is_available() else "cpu")


    if agent_opponent is None:
        agent_opponent = RandomPolicy(action_space=env.action_space)
        
    if agent_type=="qostaushy":
        agents = [agent_opponent, agent_train]
    else:
        agents = [agent_train, agent_opponent]
    policy = MultiAgentPolicyManager(policies=agents, env=env)
    return policy, optim, env.agents

In [12]:
def play_vs_random(agent, type, seed=777):
    
    env = _get_env()

    rand_policy = RandomPolicy(action_space=env.action_space)
    env.action_space.seed(seed)

    if type == 'qostaushy':
        policies = MultiAgentPolicyManager(policies=[rand_policy, agent], env=env)
    else:
        policies = MultiAgentPolicyManager(policies=[agent, rand_policy], env=env)

    env = DummyVectorEnv([lambda: env])

    collector = Collector(policies, env)

    result = collector.collect(n_episode=100)#, reset_before_collect=True)
    
    print(f'Agent {type} vs random', PLAYS)

In [13]:
def play_vs_others(agent, type, opponents):

    for n_agent_opp, agent_opponent in enumerate(opponents):
        
        env = _get_env()#render_mode='human')

        if type == "bastaushy":

            policies = MultiAgentPolicyManager(policies=[agent, agent_opponent], env=env)
            
        else:
            
            policies = MultiAgentPolicyManager(policies=[agent_opponent, agent], env=env)
        
        test_envs = DummyVectorEnv([lambda: env for _ in range(10)])

        collector = Collector(policies, test_envs, exploration_noise=False)

        result = collector.collect(n_episode=1)
        
    print(f'{type} plays vs others: {PLAYS}')

## 🐧 Training agents

### 🦚 Method train agent

In [14]:
def train_agent_dqn(index, net_arch, agent_idx, agent_train=None, agent_opponent=None, seed=777):
    # Before evaluate this cell run the cell with env
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(TRAIN_NUM)])
    test_envs = DummyVectorEnv([_get_env for _ in range(TEST_NUM)])
    
    # seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)
    
    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents_dqn(agent_train=agent_train, agent_opponent=agent_opponent, agent_type=agent_idx2name[agent_idx], net_arch=net_arch)
    
    # # ======== Step 3: Collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(100_000, len(train_envs)),
        exploration_noise=True,
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)

    train_collector.collect(n_step=BATCH_SIZE * TRAIN_NUM)  # batch size * training_num
    log_path = os.path.join(LOGS_PATH, str(index))
    writer = SummaryWriter(log_path)
    logger = TensorboardLogger(writer)
    
    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):
        model_save_path = os.path.join(MODELS_PATH, f'policy_dqn_{arch2str(net_arch)}_{index}.pth')
        os.makedirs(MODELS_PATH, exist_ok=True)
        
        torch.save(policy.policies[agents[agent_idx]].state_dict(), model_save_path)
        global PLAYS
        PLAYS ["bastaushy"], PLAYS["qostaushy"] = 0, 0
        net = Net(
            state_shape=(23,),
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=net_arch,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        
        agent_trained = DQNPolicy(
            model=net,
            optim = torch.optim.Adam(net.parameters(), lr=LR),
            discount_factor=DF,
            estimation_step=3,
            target_update_freq=320,
            action_space=env.action_space,
            observation_space=env.observation_space
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        agent_trained.load_state_dict(torch.load(f"{MODELS_PATH}/policy_dqn_{arch2str(net_arch)}_{index}.pth", map_location=torch.device('cpu')))
        play_vs_random(agent_trained, agent_idx2name[agent_idx], seed=seed)
        # PLAYS ["bastaushy"], PLAYS["qostaushy"] = 0, 0
        # play_vs_others(agent_trained, agent_idx2name[agent_idx], agents_trained[(agent_idx + 1) % 2 : len(agents_trained) : 2])
    
    def stop_fn(mean_rewards):
        return mean_rewards >= 160
    
    def train_fn(epoch, env_step):
        policy.policies[agents[agent_idx]].set_eps(get_train_eps(epoch))
    
    def test_fn(epoch, env_step):
        policy.policies[agents[agent_idx]].set_eps(TEST_EPS)
    
    def reward_metric(rews):
        return rews[:, agent_idx]
    
    def get_train_eps(epoch):
        return max(0.01, 1.0 - epoch / MAX_EPOCHS)

    def get_test_eps(epoch):
        return 0.05  # Постоянное значение для тестирования

    # ======== Step 5: Run the trainer =========
    result = OffpolicyTrainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=MAX_EPOCHS,
        step_per_epoch=STEP_PER_EPOCH,
        step_per_collect=STEP_PER_COLLECT,
        episode_per_test=EPISODE_PER_TEST,
        batch_size=BATCH_SIZE,
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=UPDATE_PER_STEP,
        test_in_train=False,
        reward_metric=reward_metric,
        verbose=True,
        show_progress=True,
        logger=logger
    ).run()
    
    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print(f"\n(the trained policy can be accessed via policy.policies[agents[{agent_idx}]])")
    return policy.policies[agents[agent_idx]]

# 🐑 Self-play training

### 🐸 Download previous models

In [15]:
# urls = ["https://drive.google.com/uc?id=1gU6xKG0OsC9PnJ7v2nCqAh9p-U6rco4u",
#         "https://drive.google.com/uc?id=1El8_10upA_es-tyqQfGxzdHfvBEqIkc0",  
#         "https://drive.google.com/uc?id=1rGM4TK_QnjWdToy7fkUPmdgL0ybh4w3L",
#         "https://drive.google.com/uc?id=14bwcfNbiOVlNCpIguqpfCcHeeXtpZvsJ", 
#        ]

# for idx, url in enumerate(urls):
#     output = f'models/policy_dqn_2048x4096x4096x2048_{idx}.pth'
#     gdown.download(url, output, quiet=False);

### 🐊 Load trained models

In [16]:
# import copy

# env = _get_env()

# agents_trained = []

# net_arch = [2048, 4096, 4096, 2048]
# for idx in range(len(urls)):

#     agent_idx = idx % 2
#     net = Net(
#             state_shape=(23,),
#             action_shape=env.action_space.shape or env.action_space.n,
#             hidden_sizes=net_arch,
#             device="cuda" if torch.cuda.is_available() else "cpu"
#             )
            
#     agent_trained1 = DQNPolicy(
#             model=net,
#             optim = torch.optim.Adam(net.parameters(), lr=LR),
#             discount_factor=DF,
#             estimation_step=3,
#             target_update_freq=320,
#             action_space=env.action_space,
#             observation_space=env.observation_space
#     ).to("cuda" if torch.cuda.is_available() else "cpu")
    
#     arch_name = arch2str(net_arch)
#     agent_trained1.load_state_dict(torch.load(f"{MODELS_PATH}/policy_dqn_{arch_name}_{idx}.pth", map_location=torch.device('cpu')))
#     agent_trained = copy.deepcopy(agent_trained1)
#     agents_trained.append(agent_trained)

### 🪲 Play trained models

In [17]:
# for i in range(len(agents_trained)):
    
#     agent_idx = i % 2
    
#     PLAYS = {"bastaushy": 0, "qostaushy": 0}
    
#     play_vs_others(agents_trained[i], agent_idx2name[agent_idx], agents_trained[(agent_idx + 1) % 2 : len(agents_trained) : 2])
#     print('----------------------------------------------')

### 🪲 Play trained models vs random

In [18]:
# for i in range(len(agents_trained)):
#     agent_idx = i % 2
#     # play trained agents with random policy
#     PLAYS = {"bastaushy": 0, "qostaushy": 0}
#     play_vs_random(agents_trained[i], agent_idx2name[agent_idx], seed=891)

## 🐣 Main loop

### 🐟 Set parameters 

In [19]:
net_arch = NET_ARCHS[3]

arch_name = arch2str(net_arch)

### 🐨 Self-play

In [ ]:
for i in range(2):
        agent_idx = i % 2
        ### training
        env = _get_env()
    
        net = Net(
            state_shape=(23,),
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=net_arch,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
        
        agent_trained = DQNPolicy(
            model=net,
            optim = torch.optim.Adam(net.parameters(), lr=LR),
            discount_factor=DF,
            estimation_step=3,
            target_update_freq=320,
            action_space=env.action_space,
            observation_space=env.observation_space
        ).to("cuda" if torch.cuda.is_available() else "cpu")

        if i > 1: 
            net_opp = Net(
                state_shape=(23,),
                action_shape=env.action_space.shape or env.action_space.n,
                hidden_sizes=net_arch,
                device="cuda" if torch.cuda.is_available() else "cpu"
            )
        
            agent_trained_opp = DQNPolicy(
                model=net_opp,
                optim = torch.optim.Adam(net.parameters(), lr=LR),
                discount_factor=DF,
                estimation_step=3,
                target_update_freq=320,
                action_space=env.action_space,
                observation_space=env.observation_space
            ).to("cuda" if torch.cuda.is_available() else "cpu")

            net_prev = Net(
                state_shape=(23,),
                action_shape=env.action_space.shape or env.action_space.n,
                hidden_sizes=net_arch,
                device="cuda" if torch.cuda.is_available() else "cpu"
            )
        
            agent_trained_prev = DQNPolicy(
                model=net_prev,
                optim = torch.optim.Adam(net.parameters(), lr=LR),
                discount_factor=DF,
                estimation_step=3,
                target_update_freq=320,
                action_space=env.action_space,
                observation_space=env.observation_space
            ).to("cuda" if torch.cuda.is_available() else "cpu")
            
            ###load policy models

            idx_opp = random.choice([i for i in range((i + 1) % 2, len(agents_trained), 2)])
            agent_trained_opp.load_state_dict(torch.load(f"{MODELS_PATH}/policy_dqn_{arch_name}_{idx_opp}.pth", map_location=torch.device('cpu')))
            agent_trained_prev.load_state_dict(torch.load(f"{MODELS_PATH}/policy_dqn_{arch_name}_{i-2}.pth", map_location=torch.device('cpu')))

            print(f'{agent_idx2name[agent_idx]} is training vs {agent_idx2name[(agent_idx + 1) % 2]} {idx_opp}')
            #train_agent_dqn(index=i, net_arch=net_arch, agent_idx=agent_idx, agent_train=agent_trained_prev, agent_opponent=agent_trained_opp)
            train_agent_dqn(index=i, net_arch=net_arch, agent_idx=agent_idx, agent_opponent=agent_trained_opp, seed=SEED)
        else:
            train_agent_dqn(index=i, net_arch=net_arch, agent_idx=agent_idx)

        # agent_trained.load_state_dict(torch.load(f"{MODELS_PATH}/policy_dqn_{arch_name}_{i}.pth", map_location=torch.device('cpu')))
        # agent_trained1 = copy.deepcopy(agent_trained)
        # agents_trained.append(agent_trained1)

<ipython-input-14-1aaf7da17f55>:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent_trained.load_state_dict(torch.load(f"{MODELS_PATH}/policy_dqn_{arch2str(net_arch)}_{i

Agent bastaushy vs random {'bastaushy': 11, 'qostaushy': 89}


Epoch #1: 10001it [00:03, 2813.73it/s, bastaushy/loss=112.376, env_step=10000, len=135, n/ep=17, n/st=1000, rew=-6.47]                           
<ipython-input-14-1aaf7da17f55>:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub 

Agent bastaushy vs random {'bastaushy': 26, 'qostaushy': 74}
Epoch #1: test_reward: -61.985000 ± 74.022056, best_reward: -61.985000 ± 74.022056 in #1


Epoch #2: 10001it [00:02, 4525.38it/s, bastaushy/loss=120.483, env_step=20000, len=106, n/ep=10, n/st=1000, rew=19.00]                           


...

Agent bastaushy vs random {'bastaushy': 91, 'qostaushy': 9}
Epoch #409: test_reward: 73.750000 ± 53.761208, best_reward: 73.750000 ± 53.761208 in #409


Epoch #410: 10001it [00:02, 4566.70it/s, bastaushy/loss=74.171, env_step=4100000, len=124, n/ep=8, n/st=1000, rew=3.00]                             


Epoch #410: test_reward: 42.460000 ± 75.035448, best_reward: 73.750000 ± 53.761208 in #409


Epoch #411: 10001it [00:02, 4603.45it/s, bastaushy/loss=71.296, env_step=4110000, len=98, n/ep=8, n/st=1000, rew=46.12]                              


Epoch #411: test_reward: 41.440000 ± 77.789179, best_reward: 73.750000 ± 53.761208 in #409
